In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')
bericht_df = pd.read_csv(r'..\..\data\berichtinteracties_met_sentiment.csv', delimiter = ';')

In [ ]:
# Merge bericht_df with evenement_df
bericht_df = bericht_df.merge(evenement_df, on = "EvenementID", how = "left")

# Merge the result with gebruikers_df
bericht_df = bericht_df.merge(gebruikers_df, on = "GebruikerID", how = "left")

# Rename columns for consistency
bericht_df.rename(
    columns = {col: f"Evenement_{col}" for col in evenement_df.columns if col != "EvenementID"},
    inplace = True
)

bericht_df.rename(
    columns = {col: f"Gebruiker_{col}" for col in gebruikers_df.columns if col != "GebruikerID"},
    inplace = True
)

bericht_df

In [ ]:
# Remove nonvalid entries
bericht_df = bericht_df[~(bericht_df['Gebruiker_Leeftijd'].isna())]
bericht_df = bericht_df[~(bericht_df['Gebruiker_Lidmaatschapstype'] == 'x')]
bericht_df

In [ ]:
# Convert ID to ints
to_convert = ["BerichtID", "GebruikerID", "EvenementID", "Evenement_OrganisatorID"]

for col in to_convert:
    bericht_df[col] = bericht_df[col].str[1:].astype(int)
    
bericht_df

In [ ]:
# Convert columns to datetime datatype and extract month
date_columns = ['BerichtDatum', 'Evenement_EvenementDatum', 'Gebruiker_RegistratieDatum', 'Gebruiker_LaatsteLogin']

for col in date_columns:
    bericht_df[col] = pd.to_datetime(bericht_df[col], dayfirst = True)

for col in date_columns:
    bericht_df[f'{col}_month'] = bericht_df[col].dt.month

bericht_df

In [ ]:
# Create boolean dataframe
dummies_df = pd.DataFrame()

# Bericht columns
dummies_df["BerichtDatum_is_weekday"] = bericht_df["BerichtDatum"].dt.weekday < 5
dummies_df["BerichtDatum_is_weekend"] = bericht_df["BerichtDatum"].dt.weekday >= 5

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['BerichtSentiment'], prefix = 'BerichtSentiment')], axis = 1)

dummies_df[f'Low likes'] = bericht_df["OntvangenLikes"] < bericht_df["OntvangenLikes"].mean()
dummies_df[f'High likes'] = bericht_df["OntvangenLikes"] >= bericht_df["OntvangenLikes"].mean()

dummies_df[f'Low reacties'] = bericht_df["OntvangenReacties"] < bericht_df["OntvangenReacties"].mean()
dummies_df[f'High reacties'] = bericht_df["OntvangenReacties"] >= bericht_df["OntvangenReacties"].mean()


# Evenement columns
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Evenement_EvenementType'], prefix = 'EvenementType')], axis = 1)

dummies_df["EvenementDatum_is_weekday"] = bericht_df["Evenement_EvenementDatum"].dt.weekday < 5
dummies_df["EvenementDatum_is_weekend"] = bericht_df["Evenement_EvenementDatum"].dt.weekday >= 5

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Evenement_EvenementDatum_month'], prefix = 'EvenementDatum_month')], axis = 1)


# Gebruiker columns
dummies_df[f'Younger'] = bericht_df["Gebruiker_Leeftijd"] <= bericht_df["Gebruiker_Leeftijd"].mode()[0]
dummies_df[f'Older'] = bericht_df["Gebruiker_Leeftijd"] > bericht_df["Gebruiker_Leeftijd"].mode()[0]

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Studierichting'].replace('Data engineering', 'Data Engineering'), prefix = 'Studierichting')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Studiejaar'], prefix = 'Studiejaar')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Lidmaatschapstype'], prefix = 'Lidmaatschapstype')], axis = 1)

dummies_df

In [ ]:
# Create frequent itemsets and association rules
freq_itemsets = apriori(dummies_df, min_support = 0.1, use_colnames = True)
association_rules(freq_itemsets, metric = 'lift', min_threshold = 1.25)